## Imports

In [ ]:
%load_ext lab_black

import h5py
import os

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache
import lmfit as lm

from typing import Dict, List, Optional, Tuple
import numpy as np
import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt


import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    analysis_functions,
    plotting_functions,
    HDF5_loader,
    misc_functions,
    filter_functions,
    tr_functions,
    loading_functions,
    cnn,
)

colors = pc.qualitative.D3
colors_seq = pc.sequential.dense
angstrom = "\u212B"
Theta = "\u0398"
phi = "\u03C6"

# Load Data

In [ ]:
# ## Convert lamp data ##

# ddir = r"E:\atully\arpes_data\2023_February\Lamp"

# # STEP 1 ##
# # Convert ibw to hdf5
# fn = "Img7_Lamp_g_kw.ibw"
# HDF5_loader.ibw_to_hdf5(ddir, fn, export=True)

# # Check conversion worked
# data, theta, energy = HDF5_loader.load_hdf5(
#     ddir, "Img7_Lamp_g_kw.h5"
# )  # load data from hdf5
# data.shape, theta.shape, energy.shape

In [ ]:
title = f"Lamp on C<sub>60</sub>, May 2023"
# xaxis_title=f"k<sub>x</sub> [{angstrom}<sup>-1</sup>]",
xaxis_title = f"{Theta}"
yaxis_title = f"E<sub>K</sub> (eV)"

In [ ]:
ddir = r"E:\atully\arpes_data\2023_May\May13\C60\Lamp"

files = ["Img1.h5", "Img2.h5"]
# data, theta, energy = loading_functions.load_hdf5(ddir, file)

In [ ]:
## Limit Dataset ##

xlim = (-18, 16)
ylim = (14.2, 15.4)
# xlim = None
# ylim = None

all_vals = []
for file in files:
    data, theta, energy = loading_functions.load_hdf5(ddir, file)
    all_vals.append(analysis_functions.limit_dataset(theta, energy, data, xlim, ylim))

x_1, y_1, d_1 = all_vals[0]
x_2, y_2, d_2 = all_vals[1]

names = [
    "img1",
    "img2",
]

In [ ]:
## FFT + Limit Data ##

ddir = r"E:\atully\arpes_data\2023_May\May13\C60\Lamp"
file = ["Img2.h5"]
# file = ["UPS.h5"]

data, theta, energy = HDF5_loader.load_hdf5(
    ddir, file[0]
)  # load data from hdf5: highest stats HOMO scan or UPS scan

f_data = filter_functions.fft2d_mask(data, plot=False)

x, y, d = analysis_functions.limit_dataset(
    theta,
    energy,
    f_data,
    xlim=(-18, 16),
    ylim=(14.2, 15.4),
    # ylim=None,
)

d = analysis_functions.norm_data(d)

In [ ]:
## Plot Data ##

# x, y, d = x_1, y_1, d_1

## FFT Data --> doesn't work on x_1, y_1, d_1
# d = filter_functions.fft2d_mask(d, plot=False)

fig = tr_functions.thesis_fig(
    title=f"{title}<br>File: {file}",
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
)

fig.add_trace(
    go.Heatmap(x=x, y=y, z=analysis_functions.norm_data(d), coloraxis="coloraxis")
)

# fig.update_coloraxes(cmin=0, cmax=0.14)
fig.update_coloraxes(cmin=0, cmax=0.3)

fig.show()

In [ ]:
## FFT Dataset ##
# # Load original dataset
# fp = r"E:\atully\arpes_data\2023_May\6eV\TR"
# fn = r"TR3_Ali_avg.h5"
# data, theta, phi, energy = HDF5_loader.load_hdf5(fp, fn)  # load data from hdf5

# # Generate FFT filtered dataset
# filter_functions.generate_FFT_filtered_dataset(
#     theta=theta, energy=energy, phi=phi, data=data, fp=fp, fn=fn
# )

In [ ]:
## Get and Plot 1D Data --> EDC ##

fig = tr_functions.thesis_fig(
    title=f"EDC",
    xaxis_title=yaxis_title,
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=1000,
)

for i in range(1):
    # x, y, d = all_vals[i]
    y, col = tr_functions.get_1d_x_slice(
        x=x, y=y, data=analysis_functions.norm_data(d), ylims=None, x_range=None
    )

    # Plot Data
    color = colors[i % len(colors)]
    fig.add_trace(go.Scatter(x=y, y=col, name=names[i], line=dict(color=color)))

fig.show()

# fig.write_image(r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\prelim_EDCs.png")

In [ ]:
## Fit Data ##

# x, y, d = all_vals[1]

y_1d, col = tr_functions.get_1d_x_slice(
    x=x, y=y, data=analysis_functions.norm_data(d), ylims=None, x_range=None
)

x = y_1d
data = col

offset_type = "constant"

## Offset
c = np.mean(data)
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)

lorentz1 = fitting_functions.make_lorentzian(
    num="A_", amplitude=0.1, center=14.8, sigma=0.2
)

full_model = lorentz1 + offset

fit = full_model.fit(data, x=x)

fit.plot()

print(fit.params["iA__fwhm"])
print(fit.params["iA__center"])

In [ ]:
## Plot Fit Components on Data ##

components = fit.eval_components(x=y_1d)

fig = tr_functions.thesis_fig(
    title=f"Data + Fit Components",
    xaxis_title="Energy (eV)",
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=1000,
    dtick_y=0.02,
)


fig.add_trace(
    go.Scatter(
        x=y_1d,
        y=col - components["parabolic"],
        name=f"data - offset",
        line=dict(color=colors[0]),
    )
)


for model_name, model_value in list(components.items())[0:1]:
    fig.add_trace(
        go.Scatter(
            x=y_1d,
            y=model_value,
            name="fit",
        )
    )


for model_name, model_value in list(components.items())[0:1]:
    fig.add_annotation(
        x=fit.params[f"{model_name}center"].value,
        yref="y domain",
        y=0.1,
        showarrow=False,
        text=f'Center: {fit.params[f"{model_name}center"].value:.2f} eV<br>FWHM: {fit.params[f"{model_name}fwhm"].value:.2f} eV',
        font=dict(size=18, color=colors[0]),
        bgcolor="white",
    )

fig.show()

# February Lamp Data

In [ ]:
ddir = r"E:\atully\arpes_data\2023_February\Lamp"

file = "Img7_Lamp_g_kw.h5"

EF = 16.869

In [ ]:
## FFT + Limit Data ##

data, theta, energy = HDF5_loader.load_hdf5(ddir, file)  # load data from hdf5

f_data = filter_functions.fft2d_mask(data, plot=False)

x, y, d = analysis_functions.limit_dataset(
    theta,
    energy,
    f_data,
    xlim=(-0.6, 0.6),
    ylim=(14.2, 15.4),
    # ylim=None,
)

d = analysis_functions.norm_data(d)

In [ ]:
## Plot Data ##

fig = tr_functions.thesis_fig(
    title=f"Lamp on C<sub>60</sub>, February 2023<br>File: {file}",
    xaxis_title=f"k<sub>x</sub> ({angstrom}<sup>-1</sup>)",
    yaxis_title=yaxis_title,
    equiv_axes=False,
)

fig.add_trace(go.Heatmap(x=x, y=y, z=d, coloraxis="coloraxis"))

# fig.update_coloraxes(cmin=0, cmax=0.3)
fig.update_coloraxes(cmin=0, cmax=0.14)

fig.show()

In [ ]:
## Get and Plot 1D Data ##

fig = tr_functions.thesis_fig(
    title=f"EDC of February Lamp Data",
    xaxis_title="E<sub>K</sub> (eV)",
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=1000,
)

y_1d, col = tr_functions.get_1d_x_slice(x=x, y=y, data=d, ylims=None, x_range=None)

# Plot Data
fig.add_trace(go.Scatter(x=y_1d - EF, y=col, name="data", line=dict(color=colors[0])))

fig.show()

In [ ]:
## Fit Data ##

# x = y_1d - EF
x = y_1d
data = col

offset_type = "constant"

## Offset
c = np.mean(data)
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)

# ## Gaussian
# gauss = fitting_functions.make_gaussian(num="A_", amplitude=1, center=14.8, sigma=0.5)

## Lorentzians
lorentz1 = fitting_functions.make_lorentzian(
    # num="A_", amplitude=0.1, center=-2, sigma=0.2
    num="A_",
    amplitude=0.1,
    center=14.8,
    sigma=0.2,
)

full_model = lorentz1 + offset

fit = full_model.fit(data, x=x)

fit.plot()

print(fit.params["iA__fwhm"])
print(fit.params["iA__center"])

In [ ]:
# fit.params

In [ ]:
## Plot Fit Components on Data ##

# components = fit.eval_components(x=y_1d - EF)
components = fit.eval_components(x=y_1d)

fig = tr_functions.thesis_fig(
    title=f"Data + Fit Components",
    xaxis_title="E<sub>K</sub> (eV)",
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=1000,
)


fig.add_trace(
    go.Scatter(
        # x=y_1d - EF,
        x=y_1d,
        y=col - components["parabolic"],
        name=f"data - offset",
        line=dict(color=colors[0]),
    )
)


for model_name, model_value in list(components.items())[0:1]:
    fig.add_trace(
        go.Scatter(
            # x=y_1d - EF,
            x=y_1d,
            y=model_value,
            name="fit",
        )
    )


for model_name, model_value in list(components.items())[0:1]:
    fig.add_annotation(
        x=fit.params[f"{model_name}center"].value,
        yref="y domain",
        y=0.1,
        showarrow=False,
        text=f'Center: {fit.params[f"{model_name}center"].value:.2f} eV<br>FWHM: {fit.params[f"{model_name}fwhm"].value:.2f} eV',
        font=dict(size=18, color=colors[0]),
        bgcolor="white",
    )


# for offset, (model_name, model_value) in zip(
#     [0.06, 0.0], list(components.items())[0:2]
# ):
#     fig.add_trace(
#         go.Scatter(
#             x=y_1d,
#             # y=model_value,
#             y=model_value + offset,
#             name=model_name,
#             # line=dict(color=color[0]),
#         )
#     )

# fig.add_annotation(x=1.6, y=1.7, text=f'{params["iA__center"].value:.2f}')

fig.show()

# May 2023 FS

In [ ]:
def average_timescans(files, ddir, new_filename):
    datas = []
    for i in range(0, len(files)):
        ad = ARPES_DATA[files[i]]
        datas.append(ad.data)
    data_avg = np.mean(datas, axis=0)
    print(data_avg.shape)

    new_data = data_avg

    new_fn = os.path.join(ddir, new_filename)

    with h5py.File(
        new_fn, "w"
    ) as f:  # Note: 'w' creates a new empty file (or overwrites), use 'r+' to modify an existing file
        f["data"] = new_data.T
        axes_names = [
            "angles",
            "energies",
        ]  # Change these to match your axes labels
        axes = [ad.theta, ad.energy]
        for axis, name in zip(axes, axes_names):
            f[name] = np.atleast_2d(axis).T
        entry_group = f.require_group("entry1")
        entry_group["ScanValues"] = np.atleast_2d(ad.phi_or_time).T
    return new_fn

In [ ]:
ddir = r"E:\atully\arpes_data\2023_May\May13\C60\Lamp"

files = []
files = [f"FS_000.h5"]
files.extend([f"FS_003.h5"])

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=kx, phi_or_time=ky, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

In [ ]:
average_timescans(files, ddir, "FS_Ali_avg.h5")

In [ ]:
ddir = r"E:\atully\arpes_data\2023_May\May13\C60\Lamp"

files = []
files = [f"FS_Ali_avg.h5"]

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=kx, phi_or_time=ky, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad = ARPES_DATA[files[0]]

In [ ]:
slice_dim = "y"
# slice_val = 0
int_range = 0.05
xlim = (-18, 16)
# xlim = None
ylim = None
x_bin = 2
y_bin = 2

# Load Data

for i, v in enumerate([15.4, 15.2, 15.0, 14.8, 14.4, 14.2]):  # Lamp
    x, y, d = tr_functions.slice_datacube(
        ad_dataclass=ad,
        slice_dim=slice_dim,
        slice_val=v,
        int_range=int_range,
        xlim=xlim,
        ylim=ylim,
        x_bin=x_bin,
        y_bin=y_bin,
        norm_data=True,
        plot_data=False,
    )

    ## Plot Data Plotly
    fig = tr_functions.thesis_fig(
        title=f"E<sub>K</sub> = {v}",
        xaxis_title=f"{Theta}",
        yaxis_title=f"{phi}",
        equiv_axes=False,
        height=500,
        width=500,
    )

    fig.add_trace(
        go.Heatmap(x=x, y=y, z=analysis_functions.norm_data(d), coloraxis="coloraxis")
    )

    if xlim is not None:
        fig.update_xaxes(range=[xlim[0], xlim[1]], constrain="domain")

    fig.update_coloraxes(cmin=0, cmax=0.5)

    fig.show()